In [79]:
#imports

from string import punctuation
from os import listdir
from numpy import array
from numpy import asarray
from numpy import zeros
import nltk
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Embedding, Flatten, GlobalMaxPool1D, Dropout, LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from collections import Counter
from nltk.corpus import stopwords
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [142]:
# load data helpers

class ContstructionData:
    def __init__(self):
        self.training_data = [
            {"label":"Concrete", "sentence":"Please indicate which concrete elements get a smooth formed and rubbed finish."},
            {"label":"Concrete", "sentence":"A note in concrete section of general construction notes indicates slump to be 4\" where specification in concrete section says 5\" slump. Please verify"},
            {"label":"Concrete", "sentence":"Need demensions and specs for the precast slab cement work and what moulde type"},
            {"label":"Concrete", "sentence":"How much Slab should we pour?"},
            {"label":"Concrete", "sentence":"Please specify the flooring underlay or cementious underlayments"},
            {"label":"Concrete", "sentence":"Specify cast-in-place mix and quantity for transport"},
            {"label":"Concrete", "sentence":"Provide concrete haunch for the sidewalk slab to acheive the finish grade elevations shown in the plans. Account for theoretical deflections due to self weight, pavers, slab forming systems, etc. as required"},
            {"label":"Concrete", "sentence":"see attached plan sheet and picture of the crack in the concrete that has occurred in 3 different locations as discussed at the monthly meeting. I am concerned these cracks will get worse once the freezer is in operation. Please advise how to repair"},
            {"label":"Concrete", "sentence":"subcontractor would like to pour the slab and then epoxy in the rebar to ensure the rebar is placed in an accurate position. Please confirm that this is acceptable using Retrofit XP Epoxy"},
            {"label":"Concrete", "sentence":"Drawing 5 indicates that Concrete in Structures, Slab‐on‐Grade is to be used. Please advise which Class of concrete should be used for Concrete in Structures, Slab‐on‐Grade."},
            {"label":"Concrete", "sentence":"Should we bolt items to concrete inserts and structural elements?"},
            {"label":"Concrete", "sentence":"Can we use crushed concrete"},
            {"label":"Concrete", "sentence":"should we use a cement underlayment to level the floor"},
            {"label":"Electrical", "sentence":"It was noted that the vanity lights were left off of the drawings in some locations and shown incorrectly in others."},
            {"label":"Electrical", "sentence":"What lighting fixtures do you want?"},
            {"label":"Electrical", "sentence":"need to rework electric blueprint"},
            {"label":"Electrical", "sentence":"what type of light bulbs do you want"},
            {"label":"Electrical", "sentence":"Should we use metal barriers to separate wiring of different systems and voltage"},
            {"label":"Electrical", "sentence":"Will lighting be required on the temporary bridge? Will permanent lighting be required in the Park?"},
            {"label":"Electrical", "sentence":"Drawing 505 calls for fiber optic cable to be run from IT closet 106 to (2) other racks but does not tell us how many strands to include. Please clarify how many strands each cable should be"},
            {"label":"Electrical", "sentence":"Drawing 505 calls for LC fiber connectors, but the specifications call for SC connectors. Please clarify which is correct"},
            {"label":"Electrical", "sentence":"Indicate the available fault current and ampacity interrupting capacity for all switchboards, distribution boards, panelboards, transfer switches, enclosed circuit breakers, motor control centers, VFDs"},
            {"label":"Electrical", "sentence":"Please confirm that all pedestrian post top light fixtures, parking lot light fixtures, and roadway light fixtures shall be individually fused"},
            {"label":"Electrical", "sentence":"Please provide protective barriers around primary switchgear (Vacuum switches and PMH switchgear), transformers, electrical and communications manholes"},
            {"label":"Electrical", "sentence":"need to repair power distribution systems and components, please advice"},
            {"label":"Electrical", "sentence":"underground wiring needs rework"},
            {"label":"Finishes", "sentence":"Need to know paint color"},
            {"label":"Finishes", "sentence":"It appears that there are numerous locations where the existing painted surfaces have failed, resulting in rustication, and in some cases rot. In order to prevent further failure of these surfaces, we are recommending complete removal of the existing paint to bare steel, and then proceed with metal primer and finish coats"},
            {"label":"Finishes", "sentence":"can you specify if you want tile or carpet or wood flooring and what type"},
            {"label":"Finishes", "sentence":"how much wallpaper for the wall and what design and what color"},
            {"label":"Finishes", "sentence":"please specify Terrazzo quartz or granite chips before it is cured and how many polish coats"},
            {"label":"Finishes", "sentence":"Spec Section 123 Does not have a Paint product listed for the exposed metal deck, trusses, conduit, ductwork. What is the paint product for the Hollow metal doors and frames?"},
            {"label":"Finishes", "sentence":"Are we to assume painting this screen?"},
            {"label":"Finishes", "sentence":"what color should we paint the bathroom"},
            {"label":"Finishes", "sentence":"what shade of blue and green do you prefer"},
            {"label":"Finishes", "sentence":"please provide tile type and color and design for the kitchen surfaces"},
            {"label":"Finishes", "sentence":"do you want a gypsum wallboard"},
            {"label":"Finishes", "sentence":"do you want laminate flooring or ceramic tiles"},
            {"label":"Masonry", "sentence":"What is the dimension of the brick that returns North and terminates into the Type A Metal Panel?"},
            {"label":"Masonry", "sentence":"Verify cut masonry brick veneer and cmu (concrete masonry units)"},
            {"label":"Masonry", "sentence":"Should bullnose type concrete masonry units be used at all edges and exterior corners?"},
            {"label":"Masonry", "sentence":"Please specify mortar for base of brick"},
            {"label":"Masonry", "sentence":"Please advise - Exterior masonry wall not structurally sound, biggest concern is the structural adequacy of the existing clay tile wall that they would like us to anchor our face brick to"},
            {"label":"Masonry", "sentence":"can we screw masonry ties into this wall for our face brick?"},
            {"label":"Masonry", "sentence":"Should we provide CMU back-up? Can we use existing masonry anchors or provide new anchors? Can we reconstruct existing brick veneer?"},
            {"label":"Masonry", "sentence":"Please provide stone type cut and color"},
            {"label":"Masonry", "sentence":"need natural stone and mortar to make load-bearing and non-bearing walls"},
            {"label":"Masonry", "sentence":"brickwork needs to be redone, please advise"},
            {"label":"Masonry", "sentence":"clarify fired clay brick or stone masonry for the wall material"},
            {"label":"Masonry", "sentence":"quantity needed for heavy sliced stone roofing"},
            {"label":"Masonry", "sentence":"clarify if this should be a dry stonewall or a stonewall with mortar, and if so, please specify mortar"},
            {"label":"Masonry", "sentence":"we need more stone for the gravel path, please approve"},
            {"label":"Masonry", "sentence":"The bricks are cracked and need to be replaced, please advise"},
        ]
        self.testing_data = [
            {"label": "Concrete", "sentence": "need specs on concrete slab"},
            {"label": "Electrical", "sentence": "lighting details needed"},
            {"label": "Finishes", "sentence": "how many coats of primer and red do you want?"},
            {"label": "Masonry", "sentence": "what rocks do you want"}
        ]
        #print ("%s sentences in training data" % len(self.training_data))
        #print ("%s sentences in testing data" % len(self.testing_data))
    
    
    # ### Refactor to load from file ###
    
    @classmethod
    def load_training_data(self):
        return ContstructionData().training_data
    
    @classmethod
    def load_testing_data(self):
        return ContstructionData().testing_data
    

training_set = ContstructionData.load_training_data()
testing_set = ContstructionData.load_testing_data()



In [150]:
# Process our data sets

x_train = []
y_train = []

x_test = []
y_test = []

labels = []

# common words that are unnecessary to classify the text
ignore_words = ['note', 'noted', 'notes', 'see', 'attached', 'need', 'needed', 'can', 'please', 'verify', 'clarify', 'specify', 'provide', 'us', 'want', 'confirm', 'would', 'know', 'says', 'assume', 'indicate', 'like', 'get', 'etc', 'rework', 'used', 'use', 'due', 'self', 'much', 'work', 'correct', 'worse', 'discussed', 'monthly', 'approve', 'make', 'many', 'shown']

def process_sentence(text):
    # create array of words
    tokens = nltk.word_tokenize(text)
    # remove punctuation
    table = str.maketrans('', '', punctuation)
    tokens = [word.translate(table) for word in tokens]
    # lowercase
    tokens = [word.lower() for word in tokens]
    # remove stopwords, ex: a, if, to, or
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # remove numbers
    tokens = [word for word in tokens if word.isalpha()]
    # remove short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = [word for word in tokens if word not in ignore_words]
    return tokens

# load all docs in a directory
def process_docs():
    for record in training_set:
        if record['label'] not in labels:
            labels.append(record['label'])
    for record in training_set:
        tokens = process_sentence(record['sentence'])
        x_train.append(tokens)
        y_train.append(labels.index(record['label']))
    for record in testing_set:
        tokens = process_sentence(record['sentence'])
        x_test.append(tokens)
        y_test.append(labels.index(record['label']))
        
process_docs()

# create training set
# 15 labels/groups
num_classes=len(labels)
max_length = max([len(s) for s in x_train])
num_labels = 4
vocab_size = len(tokenizer.word_index) + 1
batch_size = 100
print(vocab_size)

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(x_train)
encoded_docs = tokenizer.texts_to_sequences(x_train)
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
Ytrain = keras.utils.to_categorical(y_train, num_classes)

# print(labels)
# print(Xtrain)
# print(Ytrain)


#create testing set

encoded_test_docs = tokenizer.texts_to_sequences(x_test)
Xtest = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
Ytest = keras.utils.to_categorical(y_test, num_classes)

# print(Xtest)
# print(Ytest)


# Word Embedding - learned and pretrained word weights for similarities
# (this can be removed if desired but note the accuracy hit)
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    if word in tokenizer.word_index.items():
        continue
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

        

267
Loaded 400000 word vectors.
267
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.078469    1.1473     -0.66295999 ... -0.65630001  0.91429001
   1.0316    ]
 [-0.46941     1.02320004  0.0052417  ... -1.47800004  0.91582
  -0.36058   ]
 ...
 [-0.17682     0.71395999 -0.97083998 ...  0.39531001  0.56410998
   0.57550001]
 [-0.28652     0.26804999 -0.22284999 ... -0.10176    -0.069737
   0.32938999]
 [-0.46108001 -0.092607   -0.15067001 ...  0.16975001 -0.042884
  -0.65521997]]


In [153]:
filter_length = 900

model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, num_classes, padding='valid', activation='relu', strides=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain,
                    epochs=20,
                    batch_size=100,
                    validation_split=0.1)

loss, acc = model.evaluate(Xtest, Ytest, verbose=1)
print('Test Accuracy: %f' % (acc*100))

model.predict(Xtest)

Model: "sequential_70"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_70 (Embedding)     (None, 27, 100)           26700     
_________________________________________________________________
dropout_57 (Dropout)         (None, 27, 100)           0         
_________________________________________________________________
conv1d_62 (Conv1D)           (None, 24, 900)           360900    
_________________________________________________________________
flatten_39 (Flatten)         (None, 21600)             0         
_________________________________________________________________
dense_64 (Dense)             (None, 4)                 86404     
Total params: 474,004
Trainable params: 447,304
Non-trainable params: 26,700
_________________________________________________________________
Train on 47 samples, validate on 6 samples
Epoch 1/20
47/47 [==============================] - 0s 4ms/step -

array([[0.3245315 , 0.01523241, 0.13453397, 0.1266425 ],
       [0.24128148, 0.57870215, 0.2991373 , 0.0730845 ],
       [0.34405774, 0.12891743, 0.46944416, 0.05460015],
       [0.5303242 , 0.4278198 , 0.43430313, 0.27796036]], dtype=float32)